In [1]:
# Importações necessárias
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Bidirectional, BatchNormalization, Flatten, Input, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import warnings

2024-10-15 15:57:56.844592: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-15 15:57:56.853389: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-15 15:57:56.863992: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-15 15:57:56.867063: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-15 15:57:56.875600: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Ignorar avisos de futuras versões
warnings.filterwarnings("ignore", category=FutureWarning, module='sklearn')

In [3]:
# Leitura dos arquivos CSV
labels_df = pd.read_csv('labels.csv')
satellite_pos_df = pd.read_csv('satellite_pos.csv')
solar_wind_df = pd.read_csv('solar_wind.csv')
sunspots_df = pd.read_csv('sunspots.csv')

In [4]:
# Definir uma data de referência para converter 'timedelta' em 'date'
reference_date = pd.Timestamp('2024-01-01')

# Convertendo a coluna 'timedelta' para datetime e criando uma nova coluna 'date'
for df in [labels_df, satellite_pos_df, solar_wind_df, sunspots_df]:
    df['timedelta'] = pd.to_timedelta(df['timedelta'])
    df['date'] = reference_date + df['timedelta']

In [5]:
# Função de pré-processamento para os dados por hora, preenchendo valores faltantes
def preprocess_hourly_data(df):
    df.set_index('date', inplace=True)
    numeric_data = df.select_dtypes(include=[np.number])  # Seleciona apenas colunas numéricas
    return numeric_data.resample('h').mean().ffill().bfill().interpolate('linear')

In [6]:
# Pré-processamento dos diferentes DataFrames
solar_wind_hourly = preprocess_hourly_data(solar_wind_df)
satellite_pos_hourly = preprocess_hourly_data(satellite_pos_df)
sunspots_hourly = preprocess_hourly_data(sunspots_df)

# Remover a coluna 'timedelta' de solar_wind_hourly, se presente
solar_wind_hourly = solar_wind_hourly.drop(columns=['timedelta'], errors='ignore')

In [7]:
# Mesclagem dos DataFrames para criar um conjunto de dados unificado
merged_df = labels_df.merge(solar_wind_hourly, on='date', how='inner')
merged_df = merged_df.merge(satellite_pos_hourly, on='date', how='inner')
merged_df = merged_df.merge(sunspots_hourly, on='date', how='inner')

In [8]:
# Dividindo os dados de acordo com os diferentes períodos de treinamento
tA = merged_df[merged_df['period'] == 'train_a']
tB = merged_df[merged_df['period'] == 'train_b']
tC = merged_df[merged_df['period'] == 'train_c']

In [9]:
# Função para treinar e avaliar o modelo
def treinar_e_avaliar(X, y):
    # Divisão dos dados em treino, teste e validação
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    # Normalização dos dados
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()

    X_train_scaled = scaler_X.fit_transform(X_train)
    X_val_scaled = scaler_X.transform(X_val)
    X_test_scaled = scaler_X.transform(X_test)

    y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
    y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

    # Ajustar a forma dos dados para o modelo LSTM
    X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
    X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

    # Definir o modelo com camadas convolucionais, LSTM, GRU e densas
    model = Sequential([
            Input(shape=(X_train.shape[1], 1)), 
            Conv1D(64, kernel_size=3, activation='relu'),
            MaxPooling1D(pool_size=2),
            Bidirectional(LSTM(128, return_sequences=True)),
            Dropout(0.3),
            BatchNormalization(),
            Bidirectional(GRU(128, return_sequences=False)),
            Dropout(0.3),
            Dense(128, activation='relu'),
            Dropout(0.2),
            Dense(1)
    ])

    optimizer = Adam(learning_rate=1e-3)
    model.compile(optimizer=optimizer, loss='mse')

    # Callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)


    history = model.fit(X_train_reshaped, y_train_scaled,
                        epochs=100,
                        batch_size=256,
                        validation_data=(X_test_reshaped, y_test_scaled),
                        callbacks=[early_stopping, reduce_lr])

    # Previsões
    y_pred = model.predict(X_test_reshaped)
    y_pred_rescaled = scaler_y.inverse_transform(y_pred)
    y_test_rescaled = scaler_y.inverse_transform(y_test_scaled)

    # Cálculo das métricas
    rmse_value = mean_squared_error(y_test_rescaled, y_pred_rescaled, squared=False)
    mae_value = mean_absolute_error(y_test_rescaled, y_pred_rescaled)
    r2_value = r2_score(y_test_rescaled, y_pred_rescaled)

    return rmse_value, mae_value, r2_value,model

In [10]:
# Lista de colunas para usar no modelo
colunas = [
    'bx_gse', 'by_gse', 'bz_gse', 'theta_gse', 'phi_gse',
    'bx_gsm', 'by_gsm', 'bz_gsm', 'theta_gsm', 'phi_gsm',
    'bt', 'density', 'speed', 'temperature', 'smoothed_ssn'
]

In [11]:
# Listas para armazenar os resultados
all_rmse, all_mae, all_r2 = [], [], []

In [12]:
# Loop para treinar e avaliar o modelo nos diferentes períodos
for (train_data, letra) in [(tA, 'A'), (tB, 'B'), (tC, 'C')]:
    X_train_data = train_data[colunas]
    y_train_data = train_data['dst']

    rmse, mae, r2,model = treinar_e_avaliar(X_train_data, y_train_data)
    print(f"RMSE {letra}: {rmse}")
    print(f"MAE {letra}: {mae}")
    print(f"R² {letra}: {r2}")
    
    # Armazenar os resultados
    all_rmse.append(rmse)
    all_mae.append(mae)
    all_r2.append(r2)

I0000 00:00:1729018700.093023   89986 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729018700.111355   89986 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729018700.111397   89986 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729018700.113307   89986 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729018700.113356   89986 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Epoch 1/100


2024-10-15 15:58:23.020367: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
W0000 00:00:1729018703.064935   90138 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018703.087148   90138 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018703.087784   90138 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018703.088391   90138 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018703.095137   90138 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018703.100093   90138 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018703.103324   90138 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018703.103989   90138 gpu_t

 7/68 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.3091

W0000 00:00:1729018704.281543   90140 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018704.282605   90140 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018704.283755   90140 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018704.284940   90140 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018704.285559   90140 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018704.286199   90140 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018704.286959   90140 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018704.287662   90140 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018704.289021   90140 gp

...

W0000 00:00:1729018833.197556   90137 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018833.198300   90137 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018833.199000   90137 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018833.199791   90137 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018833.200901   90137 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018833.201533   90137 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018833.202169   90137 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018833.202808   90137 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018833.203433   90137 gp

136/136 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 0.0857 - val_loss: 0.4491 - learning_rate: 0.0010
Epoch 2/100
  8/136 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0101

W0000 00:00:1729018833.769927   90143 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018833.770706   90143 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018833.771366   90143 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018833.771986   90143 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018833.772692   90143 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018833.773343   90143 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018833.773970   90143 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018833.774596   90143 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018833.775317   90143 gp

136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0093 - val_loss: 0.2590 - learning_rate: 0.0010
Epoch 3/100
136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0071 - val_loss: 0.1116 - learning_rate: 0.0010
Epoch 4/100
136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0060 - val_loss: 0.0270 - learning_rate: 0.0010
Epoch 5/100
136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0052 - val_loss: 0.0038 - learning_rate: 0.0010
Epoch 6/100
136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0045 - val_loss: 0.0013 - learning_rate: 0.0010
Epoch 7/100
136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0043 - val_loss: 0.0013 - learning_rate: 0.0010
Epoch 8/100
136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0038 - val_loss: 0.0020 - learning_rate: 0.0010
Epoch 9/100
136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0036 - val_loss: 0.0017 - learning_rate: 0.0010
Epoch 10/100
136/136 ━━━━━━━━━━━━━━━━━━━━ 0s 20us/step - loss: 0.0033 - val_loss: 0.0012 - learning_rate: 5.0000e-04

W0000 00:00:1729018895.450340   90141 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018895.451072   90141 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018895.451675   90141 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018895.452322   90141 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018895.453031   90141 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018895.453666   90141 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018895.454346   90141 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018895.454972   90141 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729018895.455778   90141 gp

In [13]:
#Modelo utilizado
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 13, 64)         │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 6, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_4 (Bidirectional) │ (None, 6, 256)         │       197,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 6, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 6, 256)         │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_5 (Bidirectional) │ (None, 256)            │       296,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,584,133 (6.04 MB)

 Trainable params: 527,873 (2.01 MB)

 Non-trainable params: 512 (2.00 KB)

 Optimizer params: 1,055,748 (4.03 MB)

In [14]:
# Cálculo das médias de RMSE, MAE e R²
avg_rmse = np.mean(all_rmse)
avg_mae = np.mean(all_mae)
avg_r2 = np.mean(all_r2)

In [15]:
# Exibir os resultados médios
print(f"RMSE médio de todos os blocos: {avg_rmse}")
print(f"MAE médio de todos os blocos: {avg_mae}")
print(f"R² médio de todos os blocos: {avg_r2}")

RMSE médio de todos os blocos: 17.41638508525509
MAE médio de todos os blocos: 11.963102273748193
R² médio de todos os blocos: 0.1790368443459708
